<a href="https://colab.research.google.com/github/Nazmul1932/SPL_3/blob/main/spl3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***Main Duplicate Detection App***

In [ ]:
!pip install flask-ngrok
!pip install pyngrok
!pip install glove-python-binary

In [ ]:
!ngrok authtoken 2IURrwCxP3fRHNWe4X0HczlIklw_4UqSNQMP8endQyki5TXpp

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re
import string
import warnings
warnings.filterwarnings("ignore")
import nltk
import pandas as pd
import numpy as np
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template
import pickle
from gensim.utils import simple_preprocess as SIMPLE_PREPROCESS  
from nltk.stem import WordNetLemmatizer
from gensim.parsing.preprocessing import STOPWORDS as STOP_WORDS
from nltk.corpus import stopwords
from nltk import word_tokenize
from flask import jsonify
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from scipy.spatial import distance
warnings.filterwarnings("ignore", category=DeprecationWarning)
from glove import Glove, Corpus
from gensim.models import Word2Vec,FastText
from gensim.test.utils import get_tmpfile
from glove import Glove, Corpus
import glove

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
def cleanTextFromDescriptionAndTitle(report):

  report = report.replace('\t', '')
  report = re.sub('\w*\d\w*', '', report)
  report = report.lower()
  report = report.translate(str.maketrans('', '', string.punctuation))
    
  return report

In [ ]:
def wordTokenizationAndLemmatization(report):

    new_report = []
    for token in SIMPLE_PREPROCESS(report):
        if token not in STOP_WORDS:
          if len(token) > 5:
            new_report.append(WordNetLemmatizer().lemmatize(token, pos='v'))
    return new_report

In [ ]:
for c in range(10):
  exec('topic{} = pd.read_csv("/content/drive/MyDrive/topic{}.csv", error_bad_lines=False, engine="python")'.format(c,c))
  exec("topic{}= topic{}.drop(columns=['Unnamed: 0'])".format(c,c))
  exec("topic{}['Description'] = topic{}['Description'].map(wordTokenizationAndLemmatization)".format(c,c))

In [ ]:
for i in range(10):
  exec('sent{} = []'.format(i))
  exec('x = topic{}'.format(i))
  for j in range(len(x)):
    exec('sent{}.append(topic{}.Description[{}])'.format(i,i,j))

In [ ]:
def sim_with_clusters_lda_topn(DR, n):
    vec_bow = dictionary.doc2bow(DR)
    x= ldamallet[vec_bow]
    topic = np.asarray(x)
    sim=[]
    x= topic[np.argsort(topic[:,1])[-n:][::-1],0]
    for i in range(len(x)):
        sim.append(int(x[i]))
    return sim

In [ ]:
def average_word_vectors_ft(words, model, vocabulary, num_features):
    
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.
    
    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model[word])
    
    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector
    
def averaged_word_vectorizer_ft(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    if(any(isinstance(i, list) for i in corpus)):
      features = [average_word_vectors_ft(tokenized_sentence, model, vocabulary, num_features)
                      for tokenized_sentence in corpus]
      return np.array(features)
    else:
      features = average_word_vectors_ft(corpus, model, vocabulary, num_features)
      return np.array(features)

In [ ]:
def average_word_vectors_word2vec(words, model, vocabulary, num_features):
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.
    
    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model[word])
    
    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector

def averaged_word_vectorizer_word2vec(corpus, model, num_features):
    vocabulary = set(model.wv.vocab)
    if(any(isinstance(i, list) for i in corpus)):
      features = [average_word_vectors_word2vec(tokenized_sentence, model, vocabulary, num_features)
                      for tokenized_sentence in corpus]
      return np.array(features)
    else:
      features = average_word_vectors_word2vec(corpus, model, vocabulary, num_features)
      return np.array(features)

In [ ]:
def average_word_vectors_glove(words, model, vocabulary, num_features):  
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.  

    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model.word_vectors[model.dictionary[word]])

    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector
    


def averaged_word_vectorizer_glove(corpus, model, num_features):
    vocabulary = set(model.dictionary)
    if(any(isinstance(i, list) for i in corpus)):
      features = [average_word_vectors_glove(tokenized_sentence, model, vocabulary, num_features)
                      for tokenized_sentence in corpus]
      return np.array(features)
    else:
      features = average_word_vectors_glove(corpus, model, vocabulary, num_features)
      return np.array(features)

In [ ]:
def sim(vec1, vec2): 
  sim1 = 1/(1+distance.euclidean(vec1, vec2))
  sim2 = cosine_similarity(vec1, vec2)
  sim=(sim1+sim2)/2 
  return sim

In [ ]:
def fusion3(vec1, vec2, vec3, vec4, vec5, vec6, fusion_no):
    if (fusion_no == '4'):
        master = np.concatenate((vec1, vec2, vec3), axis=1)
        pca = PCA(n_components=100)
        master = pca.fit_transform(master)
        vec_duplicate = np.concatenate((vec4, vec5, vec6), axis=0)
        vec_duplicate=[vec_duplicate]
        vec_duplicate = pca.transform(vec_duplicate)
        return vec_duplicate, master

    else:
        raise ValueError('Invalid value for fusion')

In [ ]:

def feature_vectors_multi_modality(DR, corpus, model1, model2, model3, fusion_no):

    master_ft = averaged_word_vectorizer_ft(corpus=sent, model=model1, num_features=100)
    master_glove = averaged_word_vectorizer_glove(corpus=sent, model=model2, num_features=100)
    master_w2v = averaged_word_vectorizer_word2vec(corpus=sent, model=model3, num_features=100)

    vec_duplicate = averaged_word_vectorizer_ft(corpus=DR, model=model1, num_features=100)
    vec_duplicate = averaged_word_vectorizer_glove(corpus=DR, model=model2, num_features=100)
    vec_duplicate = averaged_word_vectorizer_word2vec(corpus=DR, model=model3, num_features=100)

    vec_duplicate , master = fusion3(master_ft,  master_glove, master_w2v, vec_duplicate,  vec_duplicate, vec_duplicate, fusion_no)
    

    return vec_duplicate,master

In [ ]:
url = '/content/drive/MyDrive/result_reports.csv'
df = pd.read_csv(url)
df

In [ ]:
def compare_topn(model1, model2, model3, sent, DR, topn, modal, fusion_no):
    
    similarity=[]

    if (modal == 'multi'):
        vec_duplicate, master= feature_vectors_multi_modality(DR, sent, model1, model2, model3, fusion_no)
    else:
        raise ValueError('Invalid Modality entered')

    for doc in range(len(master)):
        vec_master = master[doc]
        vec_master= [vec_master]
        unified_sim = sim(vec_duplicate, vec_master)

        similarity.append(unified_sim)

    similarity = np.asarray(similarity)
    similarity= np.concatenate(similarity, axis=0 )
    similarity= np.concatenate(similarity, axis=0 )
    max_similar_reports=similarity.argsort()[-topn:][::-1]

    return(max_similar_reports)

In [ ]:
for mod in range(10):
  exec('w2vmodel{} = Word2Vec.load("/content/drive/MyDrive/word2vec_ldamallet{}.model")'.format(mod, mod))
  exec('ftmodel{} = FastText.load("/content/drive/MyDrive/ftmodel_ldamallet{}.model")'.format(mod, mod))
  exec('glove{} = Glove.load("/content/drive/MyDrive/glove_ldamallet{}.model")'.format(mod, mod))

In [ ]:
def recommendations(max_cluster, sample):
        for max in max_cluster:             
            exec('model1 = FastText.load("/content/drive/MyDrive/ftmodel_ldamallet{}.model")'.format(max))              
            exec('model2 = Glove.load("/content/drive/MyDrive/glove_ldamallet{}.model")'.format(max))   
            exec('model3 =  Word2Vec.load("/content/drive/MyDrive/word2vec_ldamallet{}.model")'.format(max))                
            exec('sent = topic{}.Description'.format(max)) 
            exec('cluster = topic{}'.format(max))  

            topn = 10                                          
            fusion_no = '4'   
            modal = 'multi'
            max_sim = compare_topn(model1, model2, model3, sent, sample, topn, modal, fusion_no)  
            max_sim = max_sim.tolist()

        return max_sim 

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

ldamallet = pickle.load(open("/content/drive/MyDrive/ldamallet.pkl", "rb"))
dictionary = pickle.load(open("/content/drive/MyDrive/dictionary.pickle", "rb"))

@app.route("/", methods =["GET", "POST"])
def get_recommendation():

    if request.method == "POST":
       
        bug_report = request.form.get("bug_report")
        sample = cleanTextFromDescriptionAndTitle(bug_report)
        sample = wordTokenizationAndLemmatization(sample)
        max_cluster = sim_with_clusters_lda_topn(sample, 1)
        max_sim = recommendations(max_cluster, sample)
        return jsonify(max_sim)
    return render_template("index.html")

if __name__ == '__main__':
    app.run()
